In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm

In [ ]:
#ensure that all the columns are read as string
dt = {}
for i in range(6):
    for j in range(6):
        dt[str(i)+str(j)] = str

#load dataframe
df = pd.read_csv("../input/mlware23/dataset/train/train_landmarks.csv", dtype = dt)

In [ ]:
df.head(5)

In [ ]:
#a function to rearrange the puzzle pieces to their right positions
def rearrange(df, idx):
    img_name = df.loc[idx, "image"]
    im = Image.open("../input/mlware23/dataset/train/landmarks/" + img_name)
    im = np.array(im)
    new_im = np.zeros_like(im)
    cut = im.shape[0]//6
    for i in range(6):
        for j in range(6):
            r, c = int(df.loc[idx, str(i)+str(j)][0]), int(df.loc[idx, str(i)+str(j)][1])
            new_im[r*cut:(r+1)*cut, c*cut:(c+1)*cut] = im[i*cut:(i+1)*cut, j*cut:(j+1)*cut]
    return im, new_im

In [ ]:
puzzle, solved = rearrange(df, 1)

plt.figure()
f, axarr = plt.subplots(1,2) 
axarr[0].imshow(puzzle)
axarr[0].set_title("puzzle")
axarr[1].imshow(solved)
axarr[1].set_title("ground_truth")

In [ ]:
#train test split
n = len(df)
val_split = 0.2
val_df = df[:int(n*val_split)]
train_df = df[int(n*val_split):]

In [ ]:
# a function that loads the all the images directly
# each image is cut into 36 puzzle pieces and each piece is stored as a seperate array
def load_images(df):
    ret = []
    for idx in tqdm(df.index):
        pieces = []
        img_name = df.loc[idx, "image"]
        img = Image.open("../input/mlware23/dataset/train/landmarks/"+img_name)
        img = np.array(img)
        for i in range(6):
            for j in range(6):
                pieces.append(img[i*50:(i+1)*50, j*50:(j+1)*50])  
        ret.append(pieces)
    return np.array(ret)

In [ ]:
train_img = load_images(train_df)
val_img = load_images(val_df)

In [ ]:
print(train_img.shape)
print(val_img.shape)
#2400 puzzles, 36 pieces, each piece is a 50x50x3 image

In [ ]:
'''
a function that one hot encodes the given labels.
we have a total of 36 puzzle pieces and 36 possible positions for each of those pieces
if piece at index i has its correct position at index j, we make label[i][j] = 1 
and all the other values in label[i] are set to 0

we iterate through all positions using i and j.
for each position, its index out of 36 will be i*6+j 
and same for the value in that column
'''
def load_labels(df):
    ret = []
    for idx in tqdm(df.index):
        label = np.zeros(shape=(36,36))
        for i in range(6):
            for j in range(6):
                place = i*6 + j
                val = int(df.loc[idx, str(i)+str(j)][0])*6 + int(df.loc[idx, str(i)+str(j)][1])
                label[place][val] = 1
        ret.append(label)
    return np.array(ret)

In [ ]:
train_labels = load_labels(train_df)
val_labels = load_labels(val_df)

In [ ]:

print(train_labels.shape)
print(val_labels.shape)
#2400 puzzles, 36x36 one hot encoded matrix as output

In [ ]:
# #Modified Model

# inp = tf.keras.layers.Input(shape=(36,50,50,3)) #input all puzzle pieces at once

# #here we define the layers and feed the input later
# conv1 = tf.keras.layers.Conv2D(64, (5,5), strides=5, activation='relu')(inp)
# conv2 = tf.keras.layers.Conv2D(128, (5,5), strides=5, activation='relu')(conv1)
# flat = tf.keras.layers.Flatten()(conv2)
# dense1 = tf.keras.layers.Dense(128, activation='relu')(flat)
# dropout=tf.keras.layers.Dropout(rate=0.3)(dense1)

# #now we feed each puzzle piece one by one to the layers and store the encodings
# mid = []
# for i in range(36):
#     x = conv1(inp[:,i, :, :, :]) #first dimension is batch_size
#     x = conv2(x)
#     x = flat(x)
#     x = dense1(x)
#     mid.append(x)


# x = tf.keras.layers.concatenate(mid, axis=-1) #the encodings of all the pieces are concatenated
# x = dropout(x)
# x = tf.keras.layers.Reshape((36,36))(x) #the output is reshaped into a 36x36 output
# x = tf.keras.layers.Softmax(axis=-1)(x) #softmax to calculate probability

In [ ]:
# #Model Architechture

# inp = tf.keras.layers.Input(shape=(36,50,50,3)) #input all puzzle pieces at once

# #here we only define the layers and feed the input later
# conv1 = tf.keras.layers.Conv2D(64, (5,5), strides=5, activation='relu')
# conv2 = tf.keras.layers.Conv2D(64, (5,5), strides=5, activation='relu')
# flat = tf.keras.layers.Flatten()
# dense1 = tf.keras.layers.Dense(32, activation='relu')
# dropout=tf.keras.layers.Dropout(rate=0.3)

# #now we feed each puzzle piece one by one to the layers and store the encodings
# mid = []
# for i in range(36):
#     x = conv1(inp[:,i]) #first dimension is batch_size
#     x = conv2(x)
#     x = flat(x)
#     x = dense1(x)
#     mid.append(x)


# x = tf.keras.layers.concatenate(mid, axis=-1) #the encodings of all the pieces are concatenated
# # x = dropout(x)
# x = tf.keras.layers.Reshape((36,36))(x) #the output is reshaped into a 36x36 output
# x = tf.keras.layers.Softmax(axis=-1)(x) #softmax to calculate probability

In [ ]:
# #1.6
# inp = tf.keras.layers.Input(shape=(36,50,50,3)) #input all puzzle pieces at once

# #here we only define the layers and feed the input later
# conv0=tf.keras.layers.Conv2D(32, (3,3),strides=2, activation='relu')
# max0=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# conv1 = tf.keras.layers.Conv2D(64, (3,3), strides=2, activation='relu')
# max1=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# conv2 = tf.keras.layers.Conv2D(128, (3,3), strides=2,padding='same', activation='relu')
# max2=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# flat = tf.keras.layers.Flatten()
# dense1 = tf.keras.layers.Dense(128, activation='relu')
# dropout=tf.keras.layers.Dropout(rate=0.3)

# #now we feed each puzzle piece one by one to the layers and store the encodings
# mid = []
# for i in range(36):
     
#     x = conv0(inp[:,i]) #first dimension is batch_size
#     x=max0(x)
    
#     x = conv1(x)
#     x=max1(x)
#     x = conv2(x)
#     x=max2(x)

#     x = flat(x)
#     x = dense1(x)
#     mid.append(x)


# x = tf.keras.layers.concatenate(mid, axis=-1) #the encodings of all the pieces are concatenated
# x = dropout(x)
# x = tf.keras.layers.Reshape((36,128))(x) #the output is reshaped into a 36x36 output
# x = tf.keras.layers.Dense(36, activation='softmax')(x) #softmax to calculate probability



In [ ]:
# #Model Architechture

# inp = tf.keras.layers.Input(shape=(36,50,50,3)) #input all puzzle pieces at once

# #here we only define the layers and feed the input later
# conv0=tf.keras.layers.Conv2D(32, (5,5),strides=5, activation='relu')
# max0=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# conv1 = tf.keras.layers.Conv2D(64, (5,5), strides=5, activation='relu')
# max1=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# conv2 = tf.keras.layers.Conv2D(128, (5,5), strides=5,padding='same', activation='relu')
# max2=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# conv3 = tf.keras.layers.Conv2D(256, (5,5), strides=5,padding='same', activation='relu')
# max3=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# flat = tf.keras.layers.Flatten()
# dense1 = tf.keras.layers.Dense(36, activation='relu')

# #now we feed each puzzle piece one by one to the layers and store the encodings
# mid = []
# for i in range(36):
     
#     x = conv0(inp[:,i]) #first dimension is batch_size
#     x=max0(x)
    
#     x = conv1(x)
#     x=max1(x)
#     x = conv2(x)
#     x=max2(x)

#     x = flat(x)
#     x = dense1(x)
#     mid.append(x)


# x = tf.keras.layers.concatenate(mid, axis=-1) #the encodings of all the pieces are concatenated
# x = tf.keras.layers.Reshape((36,36))(x) #the output is reshaped into a 36x36 output
# x = tf.keras.layers.Softmax(axis=-1)(x) #softmax to calculate probability

In [ ]:
# #Model Architechture - 1.4

# inp = tf.keras.layers.Input(shape=(36,50,50,3)) #input all puzzle pieces at once

# #here we only define the layers and feed the input later
# conv0=tf.keras.layers.Conv2D(32, (5,5),strides=5, activation='elu')
# max0=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# dropout0=tf.keras.layers.Dropout(rate=0.3)

# conv1 = tf.keras.layers.Conv2D(64, (5,5), strides=5, activation='elu')
# max1=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# dropout1=tf.keras.layers.Dropout(rate=0.3)
# conv2 = tf.keras.layers.Conv2D(128, (5,5), strides=5,padding='same', activation='elu')
# max2=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# dropout2=tf.keras.layers.Dropout(rate=0.3)
# flat = tf.keras.layers.Flatten()
# dense1 = tf.keras.layers.Dense(36, activation='elu')

# #now we feed each puzzle piece one by one to the layers and store the encodings
# mid = []
# for i in range(36):
     
#     x = conv0(inp[:,i]) #first dimension is batch_size
#     x=max0(x)
#     x=dropout0(x)
#     x = conv1(x)
    
#     x=max1(x)
#     x=dropout1(x)
#     x = conv2(x)
#     x=max2(x)
#     x=dropout2(x)

#     x = flat(x)
#     x = dense1(x)
#     mid.append(x)


# x = tf.keras.layers.concatenate(mid, axis=-1) #the encodings of all the pieces are concatenated
# x = tf.keras.layers.Reshape((36,36))(x) #the output is reshaped into a 36x36 output
# x = tf.keras.layers.Softmax(axis=-1)(x) #softmax to calculate probability

In [ ]:
# inp = tf.keras.layers.Input(shape=(36,50,50,3)) #input all puzzle pieces at once

# #here we only define the layers and feed the input later
# conv0=tf.keras.layers.Conv2D(32, (5,5),strides=3, activation='elu')
# max0=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# dropout0=tf.keras.layers.Dropout(rate=0.3)

# conv1 = tf.keras.layers.Conv2D(64, (5,5), strides=3, activation='elu')
# max1=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# dropout1=tf.keras.layers.Dropout(rate=0.3)
# conv2 = tf.keras.layers.Conv2D(128, (5,5), strides=3,padding='same', activation='elu')
# max2=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# dropout2=tf.keras.layers.Dropout(rate=0.3)
# conv3 = tf.keras.layers.Conv2D(256, (5,5), strides=3,padding='same', activation='elu')
# max3=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# dropout3=tf.keras.layers.Dropout(rate=0.3)
# flat = tf.keras.layers.Flatten()
# dense1 = tf.keras.layers.Dense(36, activation='elu')

# #now we feed each puzzle piece one by one to the layers and store the encodings
# mid = []
# for i in range(36):
     
#     x = conv0(inp[:,i]) #first dimension is batch_size
#     x=max0(x)
#     x=dropout0(x)
#     x = conv1(x)
    
#     x=max1(x)
#     x=dropout1(x)
#     x = conv2(x)
#     x=max2(x)
#     x=dropout2(x)
#     x = conv3(x)
#     x=max3(x)
#     x=dropout3(x)

#     x = flat(x)
#     x = dense1(x)
#     mid.append(x)


# x = tf.keras.layers.concatenate(mid, axis=-1) #the encodings of all the pieces are concatenated
# x = tf.keras.layers.Reshape((36,36))(x) #the output is reshaped into a 36x36 output
# x = tf.keras.layers.Softmax(axis=-1)(x) #softmax to calculate probability

In [ ]:
# #Model Architechture

# inp = tf.keras.layers.Input(shape=(36,50,50,3)) #input all puzzle pieces at once

# #here we only define the layers and feed the input later
# conv0=tf.keras.layers.Conv2D(32, (5,5),strides=5, activation='elu')
# max0=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# bn0 = tf.keras.layers.BatchNormalization()

# conv1 = tf.keras.layers.Conv2D(64, (5,5), strides=5, activation='elu')
# max1=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# bn1 = tf.keras.layers.BatchNormalization()
# conv2 = tf.keras.layers.Conv2D(128, (5,5), strides=5,padding='same', activation='elu')
# max2=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# bn2 = tf.keras.layers.BatchNormalization()
# flat = tf.keras.layers.Flatten()
# dense1 = tf.keras.layers.Dense(36, activation='elu')

# #now we feed each puzzle piece one by one to the layers and store the encodings
# mid = []
# for i in range(36):
     
#     x = conv0(inp[:,i]) #first dimension is batch_size
#     x=max0(x)
#     x=bn0(x)
#     x = conv1(x)
    
#     x=max1(x)
#     x=bn1(x)
#     x = conv2(x)
#     x=max2(x)
#     x=bn2(x)

#     x = flat(x)
#     x = dense1(x)
#     mid.append(x)


# x = tf.keras.layers.concatenate(mid, axis=-1) #the encodings of all the pieces are concatenated
# x = tf.keras.layers.Reshape((36,36))(x) #the output is reshaped into a 36x36 output
# x = tf.keras.layers.Softmax(axis=-1)(x) #softmax to calculate probability

In [ ]:
# #VGG16 pretrained model

# #Model arch


# inp = tf.keras.layers.Input(shape=(36,50,50,3)) #input all puzzle pieces at once

# #here we only define the layers and feed the input later

# # VGG16
# vgg16 = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(50,50,3))
# vgg16.trainable = True

# conv0 = tf.keras.layers.Conv2D(32, (5,5),strides=5, activation=LeakyReLU())
# max0=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# bn0 = tf.keras.layers.BatchNormalization()

# conv1 = tf.keras.layers.Conv2D(64, (5,5), strides=5, activation=LeakyReLU())
# max1=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# bn1 = tf.keras.layers.BatchNormalization()
# conv2 = tf.keras.layers.Conv2D(128, (5,5), strides=5,padding='same', activation=LeakyReLU())
# max2=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# bn2 = tf.keras.layers.BatchNormalization()
# flat = tf.keras.layers.Flatten()
# dense1 = tf.keras.layers.Dense(36, activation=LeakyReLU())

# #now we feed each puzzle piece one by one to the layers and store the encodings
# mid = []
# for i in range(36):
     
#     x = conv0(inp[:,i]) #first dimension is batch_size
#     x=max0(x)
#     x=bn0(x)
#     x = conv1(x)
    
#     x=max1(x)
#     x=bn1(x)
#     x = conv2(x)
#     x=max2(x)
#     x=bn2(x)

#     x = flat(x)
#     x = dense1(x)
#     mid.append(x)


# x = tf.keras.layers.concatenate(mid, axis=-1) #the encodings of all the pieces are concatenated
# x = tf.keras.layers.Reshape((36,36))(x) #the output is reshaped into a 36x36 output
# x = tf.keras.layers.Softmax(axis=-1)(x) #softmax to calculate probability

In [ ]:
# # Improved Model arch

# inp = tf.keras.layers.Input(shape=(36,50,50,3)) #input all puzzle pieces at once

# #here we only define the layers and feed the input later

# # VGG16
# vgg16 = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(50,50,3))
# vgg16.trainable = True

# conv0 = tf.keras.layers.Conv2D(32, (5,5),strides=5, activation=LeakyReLU())
# max0=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# bn0 = tf.keras.layers.BatchNormalization()

# conv1 = tf.keras.layers.Conv2D(64, (5,5), strides=5, activation=LeakyReLU())
# max1=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# bn1 = tf.keras.layers.BatchNormalization()
# conv2 = tf.keras.layers.Conv2D(128, (5,5), strides=5,padding='same', activation=LeakyReLU())
# max2=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# bn2 = tf.keras.layers.BatchNormalization()
# flat = tf.keras.layers.Flatten()
# dense1 = tf.keras.layers.Dense(256, activation=LeakyReLU())
# dense2 = tf.keras.layers.Dense(36, activation=LeakyReLU())

# #now we feed each puzzle piece one by one to the layers and store the encodings
# mid = []
# for i in range(36):
     
#     x = conv0(inp[:,i]) #first dimension is batch_size
#     x=max0(x)
#     x=bn0(x)
#     x = conv1(x)
    
#     x=max1(x)
#     x=bn1(x)
#     x = conv2(x)
#     x=max2(x)
#     x=bn2(x)

#     x = flat(x)
#     x = dense1(x)
#     x = dense2(x)
#     mid.append(x)


# x = tf.keras.layers.concatenate(mid, axis=-1) #the encodings of all the pieces are concatenated
# x = tf.keras.layers.Reshape((36,36))(x) #the output is reshaped into a 36x36 output
# x = tf.keras.layers.Softmax(axis=-1)(x) #softmax to calculate probability

In [ ]:
# # Revised Model Arch

# inp = tf.keras.layers.Input(shape=(36,50,50,3)) #input all puzzle pieces at once

# #here we only define the layers and feed the input later

# # VGG16
# vgg16 = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(50,50,3))
# vgg16.trainable = True

# conv0 = tf.keras.layers.Conv2D(32, (5,5),strides=5, activation=LeakyReLU())
# max0=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# bn0 = tf.keras.layers.BatchNormalization()

# conv1 = tf.keras.layers.Conv2D(64, (5,5), strides=5, activation=LeakyReLU())
# max1=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# bn1 = tf.keras.layers.BatchNormalization()
# conv2 = tf.keras.layers.Conv2D(128, (5,5), strides=5,padding='same', activation=LeakyReLU())
# max2=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# bn2 = tf.keras.layers.BatchNormalization()

# # Added Convolutional Layers
# conv3 = tf.keras.layers.Conv2D(128, (3,3), strides=2,padding='same', activation=LeakyReLU())
# max3=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# bn3 = tf.keras.layers.BatchNormalization()
# conv4 = tf.keras.layers.Conv2D(256, (3,3), strides=2,padding='same', activation=LeakyReLU())
# max4=tf.keras.layers.MaxPooling2D((2,2),padding='same')
# bn4 = tf.keras.layers.BatchNormalization()

# flat = tf.keras.layers.Flatten()
# dense1 = tf.keras.layers.Dense(256, activation=LeakyReLU())
# dense2 = tf.keras.layers.Dense(36, activation=LeakyReLU())
# #Added Dropout Layer
# dropout1 = tf.keras.layers.Dropout(0.5)

# #now we feed each puzzle piece one by one to the layers and store the encodings
# mid = []
# for i in range(36):
     
#     x = conv0(inp[:,i]) #first dimension is batch_size
#     x=max0(x)
#     x=bn0(x)
#     x = conv1(x)
    
#     x=max1(x)
#     x=bn1(x)
#     x = conv2(x)
#     x=max2(x)
#     x=bn2(x)
    
#     #Added Convolutional Layers
#     x = conv3(x)
#     x=max3(x)
#     x=bn3(x)
#     x = conv4(x)
#     x=max4(x)
#     x=bn4(x)
    
#     x = flat(x)
#     x = dense1(x)
#     x = dense2(x)
    
#     #Added Dropout Layer
#     x = dropout1(x)

#     mid.append(x)


# x = tf.keras.layers.concatenate(mid, axis=-1) #the encodings of all the pieces are concatenated
# x = tf.keras.layers.Reshape((36,36))(x) #the output is reshaped into a 36x36 output
# x = tf.keras.layers.Softmax(axis=-1)(x) #softmax to calculate probability

In [ ]:
#Model Architechture

inp = tf.keras.layers.Input(shape=(36,50,50,3)) #input all puzzle pieces at once

#here we only define the layers and feed the input later
conv0=tf.keras.layers.Conv2D(32, (5,5),strides=5, activation='elu')
max0=tf.keras.layers.MaxPooling2D((2,2),padding='same')
dropout0=tf.keras.layers.Dropout(rate=0.3)

conv1 = tf.keras.layers.Conv2D(64, (5,5), strides=5, activation='elu')
max1=tf.keras.layers.MaxPooling2D((2,2),padding='same')
dropout1=tf.keras.layers.Dropout(rate=0.3)
conv2 = tf.keras.layers.Conv2D(128, (5,5), strides=5,padding='same', activation='elu')
max2=tf.keras.layers.MaxPooling2D((2,2),padding='same')
dropout2=tf.keras.layers.Dropout(rate=0.3)
base_model = tf.keras.applications.ResNet50(input_shape=(50,50,3),include_top=False,weights='imagenet')
flat = tf.keras.layers.Flatten()
dense1 = tf.keras.layers.Dense(36, activation='elu')

#now we feed each puzzle piece one by one to the layers and store the encodings
mid = []
for i in range(36):
     
    x = conv0(inp[:,i]) #first dimension is batch_size
    x=max0(x)
    x=dropout0(x)
    x = conv1(x)
    
    x=max1(x)
    x=dropout1(x)
    x = conv2(x)
    x=max2(x)
    x=dropout2(x)

    x = flat(x)
    x = dense1(x)
    mid.append(x)


x = tf.keras.layers.concatenate(mid, axis=-1) #the encodings of all the pieces are concatenated
x = tf.keras.layers.Reshape((36,36))(x) #the output is reshaped into a 36x36 output
x = tf.keras.layers.Softmax(axis=-1)(x) #softmax to calculate probability


In [ ]:
model = tf.keras.Model(inputs=inp, outputs=x)
model.compile(optimizer="adam", loss="BinaryCrossentropy",metrics=["accuracy"])

In [ ]:
model.fit(x=train_img, 
          y=train_labels, 
          batch_size=16, 
          validation_data=[val_img, val_labels], 
          shuffle=True,
          epochs=25)


In [ ]:
pred = model.predict(val_img)
pred = np.argmax(pred, axis=-1)


In [ ]:
print(pred.shape)
print(pred)
# 600 puzzles, 36 predicted permutations


In [ ]:
print(pred[0])
#predicted permutation for 0th puzzle

In [ ]:
#formatting our predictions into pandas dataframe
pred_list = []
for i in range(pred.shape[0]):
    t = []
    for j in range(pred[i].shape[0]):
        t.append(str(pred[i][j]//6) + str(pred[i][j]%6))
    pred_list.append(t)

image_df = pd.DataFrame(val_df["image"])
label_df = pd.DataFrame(pred_list)
out = pd.concat([image_df, label_df], axis=1)
out.columns = df.columns

In [ ]:
out

In [ ]:
puzzle, predicted = rearrange(out, 1)

plt.figure()
f, axarr = plt.subplots(1,2) 
axarr[0].imshow(puzzle)
axarr[0].set_title("puzzle")
axarr[1].imshow(predicted)
axarr[1].set_title("predicted")
# axarr[1].imshow(solved)
# axarr[1].set_title("ground_truth")


In [ ]:
import os
ltest_i='../input/mlware23/dataset/test/landmarks/'
lfile=(os.listdir(ltest_i))
lret = []
print(len(lfile))
for i in (lfile):
  img=Image.open('../input/mlware23/dataset/test/landmarks/'+i)
  # print(i)
  img = np.array(img)
  pieces = []
  for i in range(6):
    for j in range(6):
      pieces.append(img[i*50:(i+1)*50, j*50:(j+1)*50])
  lret.append(pieces)



In [ ]:
ltest_img=np.array(lret)

In [ ]:
lpreddd = model.predict(ltest_img)
lpreddd = np.argmax(lpreddd, axis=-1)

In [ ]:
lpreddd

In [ ]:
lpred_listtt = []
for i in range(lpreddd.shape[0]):
    t = []
    for j in range(lpreddd[i].shape[0]):
        t.append(str(lpreddd[i][j]//6) + str(lpreddd[i][j]%6))
    lpred_listtt.append(t)

limage_dfff = pd.DataFrame(lfile)
llabel_dfff = pd.DataFrame(lpred_listtt)
louttt = pd.concat([limage_dfff, llabel_dfff], axis=1)
# outtt.columns = df.columns


In [ ]:
louttt

In [ ]:
def prediction_to_submission(df):
    df = df.copy()
    for s in ['r','c']:
        for i in range(6):
            for j in range(6):
                df[str(s)+str(i)+str(j)] = None

    for idx in df.index:
        for i in range(6):
            for j in range(6):
                df.loc[idx, 'r'+str(i)+str(j)] = int(df.loc[idx, str(i)+str(j)][0])
                df.loc[idx, 'c'+str(i)+str(j)] = int(df.loc[idx, str(i)+str(j)][1])

    columns_to_remove = []
    for i in range(6):
        for j in range(6):
            columns_to_remove.append(str(i)+str(j))

    return df.drop(columns=columns_to_remove)

In [ ]:
lsubmission = prediction_to_submission(louttt)

In [ ]:
lsubmission

In [ ]:
def accuracy(pred, labels):
    correct = 0
    for i in range(pred.shape[0]):
        for j in range(pred[i].shape[0]):
            if pred[i][j] == labels[i][j]:
                correct += 1
#     print(correct,pred.shape[0]*36)
    return correct/(pred.shape[0]*36)
vv=val_df.iloc[: , 1:]
vv=vv.to_numpy(dtype=int)
vvv=val_dff.iloc[: , 1:]
vvv=vvv.to_numpy(dtype=int)
print(accuracy(pred, vv))
print(accuracy(predd, vvv))